In [1]:

import openpyxl
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta


In [2]:
# read excel file

excel_file = "tenancy_list_30aug2022.xlsx"
# you may put validations here to check extension or file size
wb = openpyxl.load_workbook(excel_file, data_only=True)

# getting all sheets
sheets = wb.sheetnames


In [3]:
# getting a particular sheet
sheet_params = wb["params"]
sheet_data = wb["tenant"]
sheet_rental = wb["rental"]
sheet_yearly_rental = wb["yearly_rental"]
sheet_sc = wb["sc"]
sheet_yearly_sc = wb["yearly_sc"]
sheet_total = wb["total_rev"]
sheet_occ = wb["occ_rate"]

#print(sheet_params)

# getting active sheet
#active_sheet = wb.active
#print(active_sheet)


In [4]:
# reading parameters
date_start = sheet_params["C4"].value
date_end = sheet_params['C5'].value
area_rentable_office = sheet_params["C6"].value
report_sum = sheet_params["C7"].value

# set excel column name for values
# level_addr_col = 'B'        # Floor
# zone_addr_col = 'C'         # Zone
# area_addr_col = 'E'         # Area
# rental_rate_addr_col = 'H'  # Rental rate
# sc_rate_addr_col = 'I'      # SC rate
# lcd_addr_col = 'F'          # LCD
# led_addr_col = 'G'          # LED
# start_show_col = 'M'

print(f"Start Date: {date_start}")
print(f"End Date: {date_end}")
print(f"rentable Office Area: {area_rentable_office}")
print(f"Sum Report: {report_sum}")


Start Date: 2022-01-01 00:00:00
End Date: 2032-12-31 00:00:00
rentable Office Area: 59081.56
Sum Report: M


In [5]:
# generate reporting daterange based on start and end dates
my_reporting_date_range = pd.date_range(start=date_start, end=date_end, freq='MS')

# Create report dataframe using datetime index for period
df_report_rental_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])   # Rental revenue
df_report_sc_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Service charge revenue
df_report_occupancy = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Occupancy rate

# Set 'date' as the index
df_report_rental_charge = df_report_rental_charge.set_index('date')
df_report_sc_charge = df_report_sc_charge.set_index('date')
df_report_occupancy = df_report_occupancy.set_index('date')

**Note:**
Below are the service charges from 2018 until 2028

In [6]:
# Calculate how much is SC rate based on months and years
# April 2022: 75000
# April 2024: 80000
# April 2026: 85000

sc_data = [
    [datetime(2018, 4, 1), 65000.00],
    [datetime(2020, 4, 1), 70000.00],
    [datetime(2022, 4, 1), 70000.00],
    [datetime(2024, 4, 1), 75000.00],
    [datetime(2026, 4, 1), 80000.00],
    [datetime(2028, 4, 1), 85000.00]
]

df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc']).set_index('date')

In [7]:
# Read a sheet into dataframe

from itertools import islice
data = sheet_data.values
cols = next(data)[1:]
data = list(data)
#idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df_data = pd.DataFrame(data, columns=cols)

# drop rows where column Tenant is Null/NA
df_data = df_data[df_data['Tenant'].notna()]

## Code Experiment
### START

In [8]:
#df_data.loc[0, "Area"]

In [9]:
#df_data.at[0, "Area"]

In [10]:
#df_data.iloc[3:5, 0:2]

In [11]:
#df_data.iloc[3,:]

In [12]:
#df_data.iat[0,3]

### Iterating every rows

In [13]:
#for index, row in df.iterrows():
#    print(row['Floor'])

### END

In [14]:
substring = '-'

for index, row in df_data.iterrows():
    vacant = False
    
    data_area = row['Area']
    data_rental_rate = row['Rental_Rate']
    data_sc_rate = row['SC_Rate']
    tenant_name = row['Tenant']

    if (data_area == None) or (substring in str(data_area)):
        data_area = 0.0

    if (data_rental_rate == None) or (substring in str(data_rental_rate)):
        data_rental_rate = 0.0

    if (data_sc_rate == None) or (substring in str(data_sc_rate)):
        data_sc_rate = 0.0

    # calculate monthly rental and service charge
    calc_rental_charge = data_area * data_rental_rate
    calc_service_charge = data_area * data_sc_rate

    start = row["Start"]
    end = row["End"]

    if (pd.isna(end)) or (end == '-'):
        vacant = True
        end = date_end

    if not vacant:
        if (pd.isna(start)) or (start == '-'):
            start = date_start

        str_level = str(row['Floor']).split('.')[0]
        str_zone = str(row['Zone'])
    
        if (str_level == 'None'):
            str_level = 'NA'
        if (str_zone == 'None'):
            str_zone = 'NA'

        # join them
        str_sep = '-'
        str_temps = [str(str_level), str(str_zone), str(index)]
        str_column_name = str_sep.join(str_temps)

        tenant_date_range = pd.date_range(start=start, end=end, freq='MS')

        # generate rental charge
        df_tenant_rental_charge = pd.DataFrame(tenant_date_range, columns=['date']) # create new df with only 1 column called 'date' and fill with the date range from lcd to led
        df_tenant_rental_charge = df_tenant_rental_charge.set_index('date') # set 'date' as the index
        df_tenant_rental_charge[str_column_name] = calc_rental_charge # add a new column called <num> and fill all with the rental_charge value
        df_report_rental_charge = df_report_rental_charge.join(df_tenant_rental_charge, how="left")

        # generate service charge report
        df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc']).set_index('date')
        sc_date_range = pd.date_range(start=df_sc_data.index.values.min(), end=df_sc_data.index.values.max(), freq='D')
        df_sc = pd.DataFrame(sc_date_range, columns=['date'])
        df_sc = df_sc.set_index('date')

        for mydate in df_sc_data.index:
            date1 = mydate
            date2 = mydate+relativedelta(years=2)-relativedelta(days=1)
            #print('{0} - {1}'.format(date1.strftime('%Y-%m-%d'), date2.strftime('%Y-%m-%d')))
            if data_sc_rate == 0:
                df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 0
            elif data_sc_rate == 84100:
                df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 84100
            else:
                df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = df_sc_data.loc[mydate, 'sc']


        df_sc = df_sc * data_area
        tenant_date_range = pd.date_range(start=start, end=end, freq='MS')

        df_tenant_service_charge = pd.DataFrame(tenant_date_range, columns=['date'])
        df_tenant_service_charge = df_tenant_service_charge.set_index('date')
        df_tenant_service_charge = df_tenant_service_charge.join(df_sc, how='left')
        df_report_sc_charge = df_report_sc_charge.join(df_tenant_service_charge, how="left")


        # df_tenant_service_charge = pd.DataFrame(tenant_date_range, columns=['date'])
        # df_tenant_service_charge = df_tenant_service_charge.set_index('date')
        # df_tenant_service_charge[str_column_name] = service_charge
        # df_report_sc_charge = df_report_sc_charge.join(df_tenant_service_charge, how="left")

        # generate occupancy report
        df_tenant_occupancy = pd.DataFrame(tenant_date_range, columns=['date'])
        df_tenant_occupancy = df_tenant_occupancy.set_index('date')
        df_tenant_occupancy[str_column_name] = data_area
        df_report_occupancy = df_report_occupancy.join(df_tenant_occupancy, how="left")


In [15]:
# Clean dataframes
df_report_rental_charge.fillna(0, inplace=True)
df_report_sc_charge.fillna(0, inplace=True)
df_report_occupancy.fillna(0, inplace=True)


In [16]:
# sum each rows
df_report_rental_charge['sum'] = df_report_rental_charge.sum(axis=1)
df_report_sc_charge['sum'] = df_report_sc_charge.sum(axis=1)
df_report_occupancy['sum'] = df_report_occupancy.sum(axis=1)

In [17]:
# create summary report
df_sum = pd.DataFrame()
df_sum['Rental'] = df_report_rental_charge.resample(report_sum).sum()['sum']
df_sum['SC'] = df_report_sc_charge.resample(report_sum).sum()['sum']
df_sum['Total'] = df_sum.sum(axis=1)

df_sum['Occ'] = df_report_occupancy.resample(report_sum).sum()['sum']
df_sum['OccPct'] = df_sum['Occ']/area_rentable_office


In [18]:
df_sum.reset_index(inplace=True)

In [19]:
df_sum

,date,Rental,SC,Total,Occ,OccPct
0,2022-01-31,1.703865e+10,3.566118e+09,2.060477e+10,47549.745343,0.804815
1,2022-02-28,1.703865e+10,3.566118e+09,2.060477e+10,47549.745343,0.804815
2,2022-03-31,1.703865e+10,3.566118e+09,2.060477e+10,47549.745343,0.804815
3,2022-04-30,1.703865e+10,3.556493e+09,2.059514e+10,47412.235343,0.802488
4,2022-05-31,1.706410e+10,3.566132e+09,2.063023e+10,47549.945343,0.804819
...,...,...,...,...,...,...
127,2032-08-31,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
128,2032-09-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
129,2032-10-31,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
130,2032-11-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000


In [20]:
df_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    132 non-null    datetime64[ns]
 1   Rental  132 non-null    float64       
 2   SC      132 non-null    float64       
 3   Total   132 non-null    float64       
 4   Occ     132 non-null    float64       
 5   OccPct  132 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 6.3 KB


In [ ]:
df_test = df_report_rental_charge.T

In [ ]:
df = df_report_rental_charge.loc['2023']
df = df.drop(['sum'], axis = 1)
df = df.T